In [1]:
import numpy as np
import pandas as pd
import torch
import sys
import torch
import tqdm
from scipy.spatial.distance import cosine
import transformers as tfm


from sentence_transformers import SentenceTransformer

In [2]:
models_list = ['bert','bertLarge','gpt2', 'roberta', 'mpnet']
models_output_size = {'bert':768,'bertLarge':1024,'gpt2':768,'roberta':768,'mpnet':768}

In [3]:
tokenizers_func = [tfm.BertTokenizer,tfm.BertTokenizer,tfm.GPT2Tokenizer,tfm.RobertaTokenizer,tfm.MPNetTokenizer]
models_func = [tfm.BertModel,tfm.BertModel,tfm.GPT2Model,tfm.RobertaModel,tfm.MPNetModel]

In [4]:
path = '/home/bastien/Downloads/LanguageModels/'

In [5]:
tokenizers = {models_list[i] : tokenizers_func[i].from_pretrained(path+models_list[i]) for i in range(len(models_list))}
models = {models_list[i] : models_func[i].from_pretrained(path+models_list[i]) for i in range(len(models_list))}

Some weights of the model checkpoint at /home/bastien/Downloads/LanguageModels/bert were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /home/bastien/Downloads/LanguageModels/bertLarge were not used when initializing BertMode

In [6]:
contexts = ['I come from', 'He lives in', 'She moved to']
short_ctxts = ['come','lives','moved']

In [7]:
cities = pd.read_csv('./csv/worldcitiespop.csv',header=0,dtype={'AccentCity':'str', 'Region':'object'}).dropna().drop('Region',axis=1)
cities = cities[cities.Population>100000].reset_index(drop=True)

In [8]:
cities

,Country,City,AccentCity,Population,Latitude,Longitude
0,ae,abu dhabi,Abu Dhabi,603687.0,24.466667,54.366667
1,ae,dubai,Dubai,1137376.0,25.258172,55.304717
2,ae,sharjah,Sharjah,543942.0,25.357310,55.403304
3,af,baglan,Baglan,108481.0,36.130684,68.708286
4,af,gardez,Gardez,103732.0,33.597439,69.225922
...,...,...,...,...,...,...
3522,zw,gweru,Gweru,201879.0,-19.450000,29.816667
3523,zw,harare,Harare,2213701.0,-17.817778,31.044722
3524,zw,kadoma,Kadoma,100276.0,-18.350000,29.916667
3525,zw,kwekwe,Kwekwe,116332.0,-18.916667,29.816667


In [9]:
capitals = pd.read_csv('./csv/country-capitals.csv').drop(['ContinentName','CountryCode'], axis=1)
capitals

,CountryName,CapitalName,Latitude,Longitude
0,Somaliland,Hargeisa,9.550000,44.050000
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667
3,Palestine,Jerusalem,31.766667,35.233333
4,Aland Islands,Mariehamn,60.116667,19.900000
...,...,...,...,...
237,Zimbabwe,Harare,-17.816667,31.033333
238,Northern Cyprus,North Nicosia,35.183333,33.366667
239,Hong Kong,Hong Kong,22.302711,114.177216
240,British Indian Ocean Territory,Diego Garcia,-7.300000,72.400000


In [10]:
countries = pd.read_csv('./csv/countries.csv')
countries

,Name,Code,Latitude,Longitude
0,Afghanistan,AF,33.0000,65.0
1,Åland Islands,AX,60.1500,20.0
2,Albania,AL,41.0000,20.0
3,Algeria,DZ,28.0000,3.0
4,American Samoa,AS,-14.3333,-170.0
...,...,...,...,...
244,Wallis and Futuna,WF,-13.3000,-176.2
245,Western Sahara,EH,24.5000,-13.0
246,Yemen,YE,15.0000,48.0
247,Zambia,ZM,-15.0000,30.0


In [123]:
import time
for i,c in tqdm.tqdm(enumerate(models_list),desc='slt',file=sys.stdout,position=1):
    #print(i,c)
    for j in tqdm.tqdm(short_ctxts,desc='inner',unit_scale=True,file=sys.stdout,position=0):
        time.sleep(0.1)
        print(j)


inner:   0%|          | 0.00/3.00 [00:00<?, ?it/s]fav
inner:  33%|███▎      | 1.00/3.00 [00:00<00:00, 9.96it/s]lives
inner:  67%|██████▋   | 2.00/3.00 [00:00<00:00, 9.90it/s]moved
inner: 100%|██████████| 3.00/3.00 [00:00<00:00, 9.84it/s]

inner:   0%|          | 0.00/3.00 [00:00<?, ?it/s]fav
inner:  33%|███▎      | 1.00/3.00 [00:00<00:00, 9.97it/s]lives
inner:  67%|██████▋   | 2.00/3.00 [00:00<00:00, 9.86it/s]moved
inner: 100%|██████████| 3.00/3.00 [00:00<00:00, 9.84it/s]

inner:   0%|          | 0.00/3.00 [00:00<?, ?it/s]fav
inner:  33%|███▎      | 1.00/3.00 [00:00<00:00, 9.97it/s]lives
inner:  67%|██████▋   | 2.00/3.00 [00:00<00:00, 9.89it/s]moved
inner: 100%|██████████| 3.00/3.00 [00:00<00:00, 9.84it/s]

inner:   0%|          | 0.00/3.00 [00:00<?, ?it/s]fav
inner:  33%|███▎      | 1.00/3.00 [00:00<00:00, 9.96it/s]lives
inner:  67%|██████▋   | 2.00/3.00 [00:00<00:00, 9.89it/s]moved
inner: 100%|██████████| 3.00/3.00 [00:00<00:00, 9.84it/s]

inner:   0%|          | 0.00/3.00 [00:00<?,

In [11]:
model_name = 'gpt2'

token = tokenizers[model_name]
model = models[model_name]

inputs = token("Hello my dog is cute", return_tensors="pt")
outputs1 = model(**inputs)


inputs = token("Hello my cat is cute".split(), return_tensors="pt",is_split_into_words=True)
outputs2 = model(**inputs)

ind_compar = 2
cosine(outputs1.last_hidden_state[0][ind_compar].detach().numpy(), outputs2.last_hidden_state[0][ind_compar].detach().numpy())

0.0010970830917358398

In [12]:
i = 0
ind = 100
ctx=contexts[i]

for entry in cities.AccentCity :
    inputs = token(ctx+" "+entry,return_tensors="pt")
    
    #If the entry is split in multiple tokens, we need to aggregate the tensors
    entry_tokens = token([entry],is_split_into_words=True,add_special_tokens=False)['input_ids']

    expected_length = len(token([entry],is_split_into_words=True,add_special_tokens=False)['input_ids']) #number of tokens for the entry
    last_ctx_token = token(ctx,add_special_tokens=False)['input_ids'][-1] #last token of the context string
    start_index = inputs['input_ids'][0].tolist().index(last_ctx_token)+1 #first index of the entry's tensors
    if inputs['input_ids'][0][start_index:start_index+expected_length].tolist()!= entry_tokens:
        print(entry_tokens)
        print(inputs['input_ids'][0][start_index:start_index+expected_length])
        raise ValueError()


#entry_embedding = outputs.last_hidden_state[0][start_index:start_index+expected_length].mean(axis=0)

#entry_embedding.shape

In [23]:
help(tqdm.tqdm)

Help on class tqdm in module tqdm.std:

class tqdm(tqdm.utils.Comparable)
 |  tqdm(*_, **__)
 |  
 |  Decorate an iterable object, returning an iterator which acts exactly
 |  like the original iterable, but prints a dynamically updating
 |  progressbar every time a value is requested.
 |  
 |  Method resolution order:
 |      tqdm
 |      tqdm.utils.Comparable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |  
 |  __del__(self)
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, exc_type, exc_value, traceback)
 |  
 |  __hash__(self)
 |      Return hash(self).
 |  
 |  __init__(self, iterable=None, desc=None, total=None, leave=True, file=None, ncols=None, mininterval=0.1, maxinterval=10.0, miniters=None, ascii=None, disable=False, unit='it', unit_scale=False, dynamic_ncols=False, smoothing=0.3, bar_format=None, initial=0, position=None, postfix=None, unit_divisor=1000, write_bytes=None, lock_args=None, nrows=None, colour=None, delay=0, gui=False, **kwargs

In [25]:
import time
for j, entry in tqdm.tqdm(enumerate(cities.AccentCity),desc=model_name + ' : ' + short_ctxts[0], total=len(cities)):
    time.sleep(0.1)

bert : come:   4%|▍         | 147/3527 [00:15<05:45,  9.78it/s]


KeyboardInterrupt: 

In [27]:
# Cities
series = cities.AccentCity
for model_name in models_list:
    token = tokenizers[model_name]
    model = models[model_name]
    size = models_output_size[model_name]
    for i, ctx in enumerate(contexts):
        buffer_arr = np.empty((len(series),size))
        for ind,entry in tqdm.tqdm(enumerate(series),desc=model_name + ' : ' + short_ctxts[i], unit_scale=True, total=len(series)):
            inputs = token(ctx+" "+entry,return_tensors="pt")
            outputs = model(**inputs)
            
            #If the entry is split in multiple tokens, we need to aggregate the tensors
            expected_length = len(token([entry],is_split_into_words=True,add_special_tokens=False)['input_ids']) #number of tokens for the entry
            last_ctx_token = token(ctx,add_special_tokens=False)['input_ids'][-1] #last token of the context string
            start_index = inputs['input_ids'][0].tolist().index(last_ctx_token)+1 #first index of the entry's tensors

            entry_embedding = outputs.last_hidden_state[0][start_index:start_index+expected_length].mean(axis=0)
            buffer_arr[ind] = entry_embedding.detach().numpy()
    
        with open('embd_files/'+model_name+'_'+ short_ctxts[i] + '_cities.npy','wb') as f:
            np.save(file=f,arr=buffer_arr)

bert : come: 100%|██████████| 3.53k/3.53k [05:13<00:00, 11.2it/s]
bert : lives: 100%|██████████| 3.53k/3.53k [04:32<00:00, 12.9it/s]
bert : moved: 100%|██████████| 3.53k/3.53k [04:16<00:00, 13.7it/s]
bertLarge : come: 100%|██████████| 3.53k/3.53k [13:27<00:00, 4.37it/s]  
bertLarge : lives: 100%|██████████| 3.53k/3.53k [13:33<00:00, 4.34it/s]
bertLarge : moved: 100%|██████████| 3.53k/3.53k [13:38<00:00, 4.31it/s]
gpt2 : come: 100%|██████████| 3.53k/3.53k [04:28<00:00, 13.1it/s] 
gpt2 : lives: 100%|██████████| 3.53k/3.53k [04:18<00:00, 13.6it/s]
gpt2 : moved: 100%|██████████| 3.53k/3.53k [04:04<00:00, 14.4it/s]
roberta : come: 100%|██████████| 3.53k/3.53k [04:03<00:00, 14.5it/s] 
roberta : lives: 100%|██████████| 3.53k/3.53k [03:48<00:00, 15.4it/s]
roberta : moved: 100%|██████████| 3.53k/3.53k [03:54<00:00, 15.0it/s]
mpnet : come: 100%|██████████| 3.53k/3.53k [04:13<00:00, 13.9it/s]
mpnet : lives: 100%|██████████| 3.53k/3.53k [04:06<00:00, 14.3it/s]
mpnet : moved: 100%|██████████| 3.53k

In [28]:
# Capitals
series = capitals.CapitalName
for model_name in models_list:
    token = tokenizers[model_name]
    model = models[model_name]
    size = models_output_size[model_name]
    for i, ctx in enumerate(contexts):
        buffer_arr = np.empty((len(series),size))
        for ind,entry in tqdm.tqdm(enumerate(series),desc=model_name + ' : ' + short_ctxts[i], unit_scale=True, total=len(series)):
            inputs = token(ctx+" "+entry,return_tensors="pt")
            outputs = model(**inputs)
            
            #If the entry is split in multiple tokens, we need to aggregate the tensors
            expected_length = len(token([entry],is_split_into_words=True,add_special_tokens=False)['input_ids']) #number of tokens for the entry
            last_ctx_token = token(ctx,add_special_tokens=False)['input_ids'][-1] #last token of the context string
            start_index = inputs['input_ids'][0].tolist().index(last_ctx_token)+1 #first index of the entry's tensors

            entry_embedding = outputs.last_hidden_state[0][start_index:start_index+expected_length].mean(axis=0)
            buffer_arr[ind] = entry_embedding.detach().numpy()
    
        with open('embd_files/'+model_name+'_'+ short_ctxts[i] + '_capitals.npy','wb') as f:
            np.save(file=f,arr=buffer_arr)

mpnet : moved: 100%|██████████| 242/242 [00:16<00:00, 14.4it/s] 


In [29]:
# Countries
series = countries.Name
for model_name in models_list:
    token = tokenizers[model_name]
    model = models[model_name]
    size = models_output_size[model_name]
    for i, ctx in enumerate(contexts):
        buffer_arr = np.empty((len(series),size))
        for ind,entry in tqdm.tqdm(enumerate(series),desc=model_name + ' : ' + short_ctxts[i], unit_scale=True, total=len(series)):
            inputs = token(ctx+" "+entry,return_tensors="pt")
            outputs = model(**inputs)
            
            #If the entry is split in multiple tokens, we need to aggregate the tensors
            expected_length = len(token([entry],is_split_into_words=True,add_special_tokens=False)['input_ids']) #number of tokens for the entry
            last_ctx_token = token(ctx,add_special_tokens=False)['input_ids'][-1] #last token of the context string
            start_index = inputs['input_ids'][0].tolist().index(last_ctx_token)+1 #first index of the entry's tensors

            entry_embedding = outputs.last_hidden_state[0][start_index:start_index+expected_length].mean(axis=0)
            buffer_arr[ind] = entry_embedding.detach().numpy()
    
        with open('embd_files/'+model_name+'_'+ short_ctxts[i] + '_countries.npy','wb') as f:
            np.save(file=f,arr=buffer_arr)

mpnet : moved: 100%|██████████| 249/249 [00:17<00:00, 14.2it/s] 
